In [0]:
import math
import torch
from torch.optim.optimizer import Optimizer, required
from torch.optim import SGD,Adam
import numpy as np  
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision
%matplotlib inline

In [0]:
trainset = torchvision.datasets.MNIST(root='',train=True, download=True)
num_classes = len(trainset.classes)
# plt.imshow(trainset.data[np.random.randint(0,len(trainset))].numpy(),cmap='gray');

In [0]:
class LeNetModel(nn.Module):
    param_conv1 = {'in_channels': 1,
            'out_channels': 8,
            'kernel_size':(5,5)}
    max_pooling_kize1 = 2

    param_conv2 = {'in_channels': 8,
            'out_channels': 16,
            'kernel_size':(5,5)}
    max_pooling_kize2 = 2
    param_fc1 = { 'in_features' :16*4*4, 'out_features':120}
    param_fc2 = { 'in_features' : 120, 'out_features': 84}
    param_fc3 = { 'in_features' : 84, 'out_features': num_classes}

    
    def __init__(self):
         super(LeNetModel, self).__init__()
         self.conv1 = nn.Conv2d(**self.param_conv1)
         self.conv2 = nn.Conv2d(**self.param_conv2)

         self.fc1 = nn.Linear(**self.param_fc1)
         self.fc2 = nn.Linear(**self.param_fc2)
         self.fc3 = nn.Linear(**self.param_fc3)

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x,self.max_pooling_kize1)

        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,self.max_pooling_kize2)

        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.fc3(x)
        x = F.relu(x)

        y = F.softmax(x,dim=1)
        return y

model_lenet = LeNetModel()  

# Optimizers
### Refrences:
SGD - https://pytorch.org/docs/master/_modules/torch/optim/sgd.html#SGD

Adam - https://github.com/pytorch/pytorch/blob/master/torch/optim/adam.py

RAdam - https://github.com/LiyuanLucasLiu/RAdam


In [0]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:                    
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

sgd = SGD(model_lenet.parameters(), lr=1e-3)
adam = Adam(model_lenet.parameters())
radam = RAdam(model_lenet.parameters())

# OR Download the (radam.py) file from github to local directory

# !wget https://raw.githubusercontent.com/LiyuanLucasLiu/RAdam/master/radam/radam.py
# from radam import RAdam
# radam = RAdam(model.parameters(),lr=0.001)

Focal Loss

![alt text](https://miro.medium.com/max/848/1*Qrw4Xcem0hnlVg2Csk2FnQ.png)


In [0]:

class FocalLoss(nn.Module):
    def __init__(self, reduction='mean', alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.reduction = reduction
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        CE_loss = F.binary_cross_entropy(inputs,
                                         targets,
                                         reduction=self.reduction)
        prob_t = self.get_prob(CE_loss)
        return self.alpha * (1-prob_t)**(self.gamma) * CE_loss

    def get_prob(self,CE_loss):
        # extract pt from -log(pt) == CE
        return torch.exp(-CE_loss)

class FocalLossWithLogits(FocalLoss):

    def __init__(self,pos_weight=None, reduction='mean', alpha=1, gamma=2):
        super(FocalLossWithLogits, self).__init__(reduction, alpha, gamma)
        self.pos_weight = pos_weight

    def forward(self, inputs, targets):
        CE_loss = F.binary_cross_entropy_with_logits(inputs,
                                                     targets,
                                                     reduction=self.reduction,
                                                     pos_weight=self.pos_weight)
        prob_t = super().get_prob(CE_loss)
        return self.alpha * (1-prob_t)**(self.gamma) * CE_loss


# Test FocalLoss


In [0]:
m = nn.Sigmoid()
loss = FocalLoss()
input_ = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input_), target)
output.backward()

# Test FocalLossWithLogits and RAdam

In [7]:
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = FocalLossWithLogits(reduction='mean')

learning_rate = 1e-4
optimizer = RAdam(model.parameters(), lr=learning_rate)
for t in range(500):

    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)


99 0.0765124037861824
199 0.01782449707388878
299 0.0032278629951179028
399 0.0006381400744430721
499 0.00015757900837343186
